## Basics of ConvNet
- Define a function that builds a `nn.Module` of a single conv block
- Defind a model (`nn.Module`) with user-defined number of specified conv layers


## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys
import re
import math
from datetime import datetime
import time
sys.dont_write_bytecode = True

In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from typing import List, Set, Dict, Tuple, Optional, Iterable, Mapping, Union, Callable, TypeVar

from pprint import pprint
from ipdb import set_trace as brpt

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torch.linalg import norm as tnorm
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import loggers as pl_loggers

# Select Visible GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## Set Path 
1. Add project root and src folders to `sys.path`
2. Set DATA_ROOT to `maptile_v2` folder

In [ ]:
this_nb_path = Path(os.getcwd())
ROOT = this_nb_path.parent
SRC = ROOT/'src'
DATA_ROOT = Path("/data/hayley-old/maptiles_v2/")
paths2add = [this_nb_path, ROOT]

print("Project root: ", str(ROOT))
print('Src folder: ', str(SRC))
print("This nb path: ", str(this_nb_path))


for p in paths2add:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(f"\n{str(p)} added to the path.")
        
print(sys.path)

In [ ]:
from collections import OrderedDict

In [ ]:
def conv_block(
    in_channels:int ,
    out_channels:int ,
    has_bn: bool=True,
    act_fn: Callable=None,
    **kwargs) -> nn.Sequential:
    """
    Returns a conv block of Conv2d -> (BN2d) -> act_fn
    
    kwargs: (will be passed to nn.Conv2d)
    - kernel_size:int
    - stride: int
    - padding
    - dilation
    - groups
    - bias
    - padding_mode
    """
    # Default conv_kwargs is overwritten by input kwargs
    conv_kwargs = {'kernel_size': 3, 'stride': 2, 'padding': 1} 
    conv_kwargs.update(kwargs)
    
    if act_fn is None:
        act_fn = nn.LeakyReLU()
    return nn.Sequential(OrderedDict([
        ('conv', nn.Conv2d(in_channels, out_channels, **conv_kwargs)),
        ('bn', nn.BatchNorm2d(out_channels) if has_bn else nn.Identity()),
        ('act', act_fn)
    ]))

def conv_blocks( 
    in_channels:int, 
    nf_list: List[int], 
    has_bn=True, 
    act_fn=None, 
    **kwargs)-> nn.Sequential:
    """
    Returns a nn.Sequential of conv_blocks, each of which is itself a nn.Sequential
    of Conv2d, (BN2d) and activation function (eg. ReLU(), LeakyReLU())
    """

    blocks = []
    nf_list.insert(0, in_channels) #in-place
    # alternatively,
    # nf_list = [in_channels, *nf_list]
    for i, (in_c, out_c) in enumerate(zip(nf_list, nf_list[1:])):
        name = f'cb{i}'
        blocks.append(
            (name, conv_block(in_c, out_c, has_bn=has_bn, act_fn=act_fn, **kwargs))
        )
        
    return nn.Sequential(OrderedDict(blocks))
                      
conv_net = conv_blocks

def test_conv_block():            
    conv_kwargs = {'kernel_size': 3, 'stride': 2, 'padding': 2}
    b1 = conv_block(3, 10, **conv_kwargs)
    print(b1)
    for name, p in b1.named_parameters():
        print(name, p.shape)
def test_conv_blocks():
    cbs = conv_blocks(3, nf_list=[5,6,7])
    for name, p in cbs.named_parameters():
        print(name, p.shape)

In [ ]:
test_conv_block();

In [ ]:
test_conv_blocks()

In [ ]:
conv_blocks(3, nf_list=[5,6,7])